In [1]:
# allow to import modules from the project root directory
import sys
import os
# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

In [2]:
import numpy as np
import random
from collections import defaultdict
from tqdm import tqdm
import pickle
from src.ParObsSnakeEnv import ParObsSnakeEnv
from src.FullObsSnakeEnv import FullObsSnakeEnv
from src.utils import compute_metrics

<img src="../../artifacts/images/Qlearning.png" alt="Q-learning algorithm" width="1000" />

In [3]:
class QLearningAgent:
    def __init__(self, env, learning_rate=0.5, discount_factor=0.99, epsilon=0.1, learning_rate_decay=0.8, epsilon_decay=0.9):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.learning_rate_decay = learning_rate_decay
        self.epsilon_decay = epsilon_decay
        self.q_table = defaultdict(lambda: np.zeros(env.action_space.n))

    def choose_action(self, state, Greedy=False):
        state = tuple(state.flatten())
        if Greedy:
            return np.argmax(self.q_table[state]), None
        if random.uniform(0, 1) < self.epsilon:
            return self.env.action_space.sample(), None  # Explore
        else:
            return np.argmax(self.q_table[state]), None  # Exploit

    def update_q_value(self, state, action, reward, next_state):
        next_action = self.choose_action(next_state, Greedy=True)  # Choose next action using epsilon-greedy approach
        next_state = tuple(next_state.flatten())
        state = tuple(state.flatten())
        td_target = reward + self.discount_factor * self.q_table[next_state][next_action]
        td_error = td_target - self.q_table[state][action]
        self.q_table[state][action] += self.learning_rate * td_error

    def print_qtable(self):
        for state, actions in self.q_table.items():
            print(f"State: {state}")
            for action, value in enumerate(actions):
                print(f"  Action {action}: {value:.2f}")
            print()
    
    def save_table(self, table_path):
        with open(table_path, 'wb') as f:
            pickle.dump(dict(self.q_table), f)
    
    def load_table(self, table_path):
        with open(table_path, 'rb') as f:
            self.q_table = pickle.load(f)

    def train(self, num_episodes):
        for episode in tqdm(range(num_episodes), desc='Training', unit='Episode'):
            state = self.env.reset()
            done = False
            while not done:
                action, _ = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)
                self.update_q_value(state, action, reward, next_state)
                state = next_state
            # decaying    
            if episode % 1000 == 0 and episode != 0:
                self.epsilon *= self.epsilon_decay  # Decay epsilon
                self.learning_rate *= self.learning_rate_decay


In [4]:
grid_size = 10
# env = FullObsSnakeEnv(grid_size=grid_size, interact=False)
env = ParObsSnakeEnv(grid_size=grid_size, interact=False)
agent = QLearningAgent(env, epsilon=0.1, discount_factor=0.9, learning_rate=0.9)

In [5]:
num_episodes = 50000
agent.train(num_episodes=num_episodes)

Training:   0%|          | 0/50000 [00:00<?, ?Episode/s]/tmp/ipykernel_63578/323638319.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.q_table[state][action] += self.learning_rate * td_error
Training: 100%|██████████| 50000/50000 [02:03<00:00, 406.09Episode/s]


In [6]:
environment = 'full 'if isinstance(env, FullObsSnakeEnv) else 'par'

table_name = f'q_learning_table_{environment}_{num_episodes}_{grid_size}.pkl'
model_weights_dir = os.path.join('../..', 'models', 'q-learning')
os.makedirs(model_weights_dir, exist_ok=True)
table_path = os.path.join(model_weights_dir, table_name)

agent.save_table(table_path)

In [7]:
if isinstance(env, ParObsSnakeEnv):
    env = ParObsSnakeEnv(grid_size=2*grid_size, interact=False)

num_simulations = 100
metrics_name = f'q_learning_metrics_{environment}_{num_episodes}_{env.grid_size}_{num_simulations}.jsn'
model_metrics_dir = os.path.join('../..', 'artifacts', 'models_stats', 'q-learning')
os.makedirs(model_metrics_dir, exist_ok=True)
metrics_path = os.path.join(model_metrics_dir, metrics_name)

compute_metrics(agent, env, metrics_path, num_simulations=num_simulations)
    

 45%|████▌     | 45/100 [00:00<00:00, 222.61it/s]

Snake length: 16, Episode reward: 1233
Snake length: 16, Episode reward: 1331
Snake length: 51, Episode reward: 4375
Snake length: 30, Episode reward: 2486
Snake length: 18, Episode reward: 1432
Snake length: 31, Episode reward: 2670
Snake length: 24, Episode reward: 1953
Snake length: 30, Episode reward: 2450
Snake length: 17, Episode reward: 1351
Snake length: 36, Episode reward: 3000
Snake length: 30, Episode reward: 2469
Snake length: 13, Episode reward: 945
Snake length: 32, Episode reward: 2755
Snake length: 19, Episode reward: 1489
Snake length: 18, Episode reward: 1446
Snake length: 28, Episode reward: 2291
Snake length: 15, Episode reward: 1177
Snake length: 37, Episode reward: 3068
Snake length: 39, Episode reward: 3340
Snake length: 16, Episode reward: 1246
Snake length: 29, Episode reward: 2412
Snake length: 35, Episode reward: 2974
Snake length: 9, Episode reward: 625
Snake length: 18, Episode reward: 1397
Snake length: 22, Episode reward: 1826
Snake length: 41, Episode re

100%|██████████| 100/100 [00:00<00:00, 244.97it/s]

Snake length: 31, Episode reward: 2605
Snake length: 5, Episode reward: 282
Snake length: 34, Episode reward: 2913
Snake length: 33, Episode reward: 2681
Snake length: 16, Episode reward: 1266
Snake length: 22, Episode reward: 1836
Snake length: 25, Episode reward: 2035
Snake length: 28, Episode reward: 2352
Snake length: 35, Episode reward: 2904
Snake length: 40, Episode reward: 3400
Snake length: 11, Episode reward: 801
Snake length: 26, Episode reward: 2188
Snake length: 25, Episode reward: 2079
Snake length: 35, Episode reward: 2899
Snake length: 31, Episode reward: 2510
Snake length: 37, Episode reward: 3089
Snake length: 38, Episode reward: 3221
Snake length: 26, Episode reward: 2127
Snake length: 17, Episode reward: 1294
Snake length: 24, Episode reward: 2042
Snake length: 20, Episode reward: 1569
Snake length: 35, Episode reward: 2939
Snake length: 35, Episode reward: 2983
Snake length: 15, Episode reward: 1172
Snake length: 45, Episode reward: 3864
Snake length: 16, Episode re

{'snake_lengths': [16,
  16,
  51,
  30,
  18,
  31,
  24,
  30,
  17,
  36,
  30,
  13,
  32,
  19,
  18,
  28,
  15,
  37,
  39,
  16,
  29,
  35,
  9,
  18,
  22,
  41,
  40,
  22,
  25,
  34,
  38,
  25,
  24,
  35,
  42,
  26,
  29,
  43,
  28,
  24,
  40,
  18,
  35,
  14,
  31,
  8,
  38,
  29,
  21,
  18,
  21,
  30,
  10,
  42,
  32,
  26,
  15,
  30,
  38,
  27,
  27,
  29,
  24,
  35,
  40,
  35,
  26,
  31,
  35,
  41,
  28,
  38,
  31,
  5,
  34,
  33,
  16,
  22,
  25,
  28,
  35,
  40,
  11,
  26,
  25,
  35,
  31,
  37,
  38,
  26,
  17,
  24,
  20,
  35,
  35,
  15,
  45,
  16,
  33,
  23],
 'episode_rewards': [1233,
  1331,
  4375,
  2486,
  1432,
  2670,
  1953,
  2450,
  1351,
  3000,
  2469,
  945,
  2755,
  1489,
  1446,
  2291,
  1177,
  3068,
  3340,
  1246,
  2412,
  2974,
  625,
  1397,
  1826,
  3558,
  3405,
  1783,
  2002,
  2900,
  3258,
  2047,
  1980,
  2971,
  3572,
  2128,
  2324,
  3598,
  2288,
  1939,
  3386,
  1441,
  2960,
  1099,
  2592,
  555,
 

In [10]:
if isinstance(env, FullObsSnakeEnv):
    env.interact = True
else:
    env = ParObsSnakeEnv(grid_size=2*grid_size)
    
state = env.reset()
done = False
while not done:
    action, _ = agent.choose_action(state)
    state, reward, done, _ = env.step(action)
    env.render()
    print(f"Reward: {reward}")

Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Re